In [48]:
## Script to calculate correlations for simulated counts from previous script

# Libraries

In [49]:
source('MS0_Libraries.r')

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
"incomplete final line found by readTableHeader on '../conda_environment/Environment_Configs.csv'"


[1] "/home/icb/corinna.losert/miniconda3/envs/scgrn_R_4_1//lib/R/library"


# Parameters

In [50]:
### Path to saved simulation results from previous script

data_path_sim = "../results/current/A1/"

In [53]:
### Parameters for simulated co-EQTL data (names of the datasets)

In [54]:
sim_data_name_counts = 'A1_4_Simulated_Counts'

In [55]:
sim_data_name_true_cor = 'A1_4_Simulation_Specified_Correlation'

In [56]:
sim_data_name_gene_summary = 'A1_4_Gene_Summary'

In [57]:
sim_data_name_cell_summary = 'A1_4_Cell_Summary'

In [ ]:
#### Simulated variant to load

In [60]:
variant = 'wijstCD4_T4' # alternatives: scaling, no scaling, 'limited_scaling

In [61]:
coeqtl_variant = 'scaling'

# Functions

In [ ]:
source('MS1_Functions.r')

# Data

## Define Links to individual datasets

In [65]:
## Function to load specified correlations
cor_data = function( ){
    path =  paste0(data_path_sim,  sim_data_name_true_cor,  variant, '_', coeqtl_variant, '.csv')
    print(file.info(path)$mtime)
    fread(path)
    

}

In [66]:
## Function to load specified counts
count_data = function(){
    path = paste0(data_path_sim,  sim_data_name_counts ,  variant, coeqtl_variant, '.csv.gz')
    print(file.info(path)$mtime)
    fread(path)
    }

In [67]:
## Function to load  specified gene summary){
get_gene_summary = function( ){
    path = paste0(data_path_sim,  sim_data_name_gene_summary  , '_', variant, '_',  coeqtl_variant, '.csv.gz')
    print(file.info(path)$mtime)
    fread(path)
    }

In [68]:
## Function to load  specified cell summary
get_cell_summary = function(){
    path = paste0(data_path_sim, sim_data_name_cell_summary, '_', variant, '_',  coeqtl_variant, '.csv.gz')
    print(file.info(path)$mtime)
    fread(path)
    }

# Get Specification of true correlation

In [69]:
### Get the specified correlation for the simulated data

In [70]:
group1_data = data.frame()

In [71]:
group1_data = cor_data()
group1_data$X = NULL
    

[1] "2025-01-31 10:46:37 CET"


Warning message in set(x, j = name, value = value):
"Tried to assign NULL to column 'X', but this column does not exist to remove"


In [72]:
head(group1_data,2)

V1,gene1,gene2,gene1_mu,gene2_mu,gene1_size,gene2_size,genotype,correlation,gene_pair,ind,ident
<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<int>,<chr>
20001,gene1_1,gene2_1,0.05645162,0.02941176,100,100,XX,0,gene1_1;gene2_1,1,XX;1
20004,gene1_4,gene2_4,0.16272190,0.23465706,100,100,XX,0,gene1_4;gene2_4,1,XX;1


In [73]:
# Different correlations in the data
unique(group1_data$correlation)

[1] 0.00 0.05 0.10

In [76]:
nrow(group1_data)

[1] 450000

In [77]:
correlation_pairs = unique(group1_data[,c('gene1', 'gene2')])

In [78]:
nrow(correlation_pairs)

[1] 10000

# Calculate the correlations for the simulated count data

In [79]:
### Empty data frame to save correlations
calc_correlations_all = data.frame()

In [80]:
### Get the count data

In [81]:
count_data = count_data()

[1] "2025-01-31 13:28:09 CET"


In [82]:
head(count_data,2)

V1,variable,cell_ident,value,scaling,mu,size,counts,basemean,ncells,ind
<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<int>,<chr>
1,gene1_1,cell_XX;1_149_1,0.9802071,0.9754055,0.05645162,100,1,0.05645162,149,XX;1
2,gene1_1,cell_XX;1_149_39,0.3085414,0.7945828,0.05645162,100,0,0.05645162,149,XX;1


In [83]:
count_data$V1 = NULL

In [84]:
### Get cell summary for merging

In [85]:
cell_summary  =get_cell_summary()

[1] "2025-01-31 12:38:40 CET"


In [86]:
head(cell_summary,2)

V1,ncells,ind,cell_ident,amount_counts,amount_genes,amount_zero_counts,mean_counts,max_counts,min_counts,var_counts,percentage_zero,total_mean_counts,scaling
<int>,<int>,<chr>,<chr>,<int>,<int>,<int>,<dbl>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,149,XX;1,cell_XX;1_149_1,7989,20000,17892,0.39945,74,0,7.101445,0.89460,0.5335534,1.335720
2,149,XX;1,cell_XX;1_149_10,9841,20000,16807,0.49205,88,0,8.085241,0.84035,0.5335534,1.084348


In [87]:
amount_cells_per_ind = unique(cell_summary[,c('ncells', 'ind')])

In [88]:
head(amount_cells_per_ind, 5)

ncells,ind
<int>,<chr>
149,XX;1
281,XX;2
271,XX;3
511,XX;4
231,XX;5


In [ ]:
### Calculate different correlations on the simulated data

In [90]:
for(i in unique(count_data$ind)){
    print(i)
    flush.console()
    ind = i
    ncells = amount_cells_per_ind$ncells[amount_cells_per_ind$ind == ind]
    # get countdata
    countdata1 = count_data[count_data$ind == i,]
    #countdata1$X = NULL
    #countdata1$variable = str_replace(countdata1$variable, '__.*', '')
    
    ### Merge cor data
    
    cor_data = merge(correlation_pairs, countdata1, by.x = c('gene1'), by.y = c('variable'))
    cor_data = merge(cor_data, countdata1, by.x = c('gene2', 'cell_ident', 'ncells', 'ind'), by.y = c('variable', 'cell_ident', 'ncells', 'ind'))
    
    ### Add cell_summary info
    cor_data = merge(cor_data, cell_summary, by = c('ncells', 'ind', 'cell_ident') ) 
    
    ## Calculate diff correlations

    calc_correlations = cor_data  %>% group_by(gene1, gene2, ncells, ind) %>% summarise(

        spearman_raw = cor(counts.x, counts.y, method = 'spearman'),

        weighted_spearman = weightedCorr(counts.x * scaling, counts.y * scaling, method = 'Spearman', weights = 1/scaling, ML = FALSE) ,
        
        weighted_spearman_non_zero_counts = weightedCorr(counts.x * scaling, counts.y * scaling, method = 'Spearman', weights = (amount_counts - amount_zero_counts), ML = FALSE),
        
        weighted_spearman_non_zero_genes = weightedCorr(counts.x * scaling, counts.y * scaling, method = 'Spearman', weights = (amount_genes - amount_zero_counts), ML = FALSE),
        
        weighted_spearman_ranks = weightedCorr(dplyr::dense_rank(counts.x * scaling), dplyr::dense_rank(counts.y * scaling), method = 'Pearson', weights = 1/scaling, ML = FALSE) ,

        pearson_raw = cor(counts.x, counts.y, method = 'pearson')  ,

        weighted_pearson = weightedCorr(counts.x * scaling, counts.y * scaling, method = 'Pearson', weights =1/scaling, ML = FALSE),
                                                  
        weighted_pearson_non_zero_counts = weightedCorr(counts.x * scaling, counts.y * scaling, method = 'Pearson', weights = (amount_counts - amount_zero_counts), ML = FALSE),
        
        weighted_pearson_non_zero_genes = weightedCorr(counts.x * scaling, counts.y * scaling, method = 'Pearson', weights = (amount_genes - amount_zero_counts), ML = FALSE),

        spearman =  cor(counts.x * scaling, counts.y * scaling, method = 'spearman'),
        
        scaled_spearman_ranks =  cor(dplyr::dense_rank(counts.x * scaling), dplyr::dense_rank(counts.y * scaling), method = 'pearson'),

        pearson =  cor((counts.x * scaling), (counts.y *scaling), method = 'pearson') ,
        

    )
    
    ## Save calculated correlations
    # V1 seperate per individual --> use if datasizes get to big
    # write.csv(calc_correlations, paste0(output_path_sc,'coeQTL_sim/', 'A1_5_Calc_Cor', ind,'_', ncells, variant, 'TRUE', coeqtl_variant, '.csv'))
    # V2: combined for all individuals
    
    calc_correlations_all = rbind(calc_correlations_all, calc_correlations) 
   
}
# write.csv(calc_correlations_all, paste0(output_path_sc,'coeQTL_sim/', 'A1_5_Calc_Cor', '_', ncells_mean ,'_', genotype, '_',  variant, coeqtl_variant, '.csv'))
 write.csv(calc_correlations_all, paste0(data_path_sim, 'A1_5_Calc_Cor_Ext',  '_',  variant, coeqtl_variant, '.csv'))   # include new versions to calculate correlation

[1] "XX;1"


Warning message:
"There were 14 warnings in `summarise()`.
The first warning was:
ℹ In argument: `spearman_raw = cor(counts.x, counts.y, method = "spearman")`.
ℹ In group 2958: `gene1 = "gene1_366"`, `gene2 = "gene2_366"`, `ncells = 149`,
  `ind = "XX;1"`.
Caused by warning in `stats::cor()`:
! the standard deviation is zero
ℹ Run `dplyr::last_dplyr_warnings()` to see the 13 remaining warnings."
`summarise()` has grouped output by 'gene1', 'gene2', 'ncells'. You can
override using the `.groups` argument.


[1] "XX;2"


Warning message:
"There were 2 warnings in `summarise()`.
The first warning was:
ℹ In argument: `spearman_raw = cor(counts.x, counts.y, method = "spearman")`.
ℹ In group 8881: `gene1 = "gene1_8991"`, `gene2 = "gene2_8991"`, `ncells =
  281`, `ind = "XX;2"`.
Caused by warning in `stats::cor()`:
! the standard deviation is zero
ℹ Run `dplyr::last_dplyr_warnings()` to see the 1 remaining warning."
`summarise()` has grouped output by 'gene1', 'gene2', 'ncells'. You can
override using the `.groups` argument.


[1] "XX;3"


...

Warning message:
"There were 2 warnings in `summarise()`.
The first warning was:
ℹ In argument: `spearman_raw = cor(counts.x, counts.y, method = "spearman")`.
ℹ In group 6295: `gene1 = "gene1_6663"`, `gene2 = "gene2_6663"`, `ncells =
  299`, `ind = "YY;8"`.
Caused by warning in `stats::cor()`:
! the standard deviation is zero
ℹ Run `dplyr::last_dplyr_warnings()` to see the 1 remaining warning."
`summarise()` has grouped output by 'gene1', 'gene2', 'ncells'. You can
override using the `.groups` argument.


[1] "YY;9"


`summarise()` has grouped output by 'gene1', 'gene2', 'ncells'. You can
override using the `.groups` argument.


[1] "YY;10"


Warning message:
"There were 2 warnings in `summarise()`.
The first warning was:
ℹ In argument: `spearman_raw = cor(counts.x, counts.y, method = "spearman")`.
ℹ In group 7973: `gene1 = "gene1_8173"`, `gene2 = "gene2_8173"`, `ncells =
  256`, `ind = "YY;10"`.
Caused by warning in `stats::cor()`:
! the standard deviation is zero
ℹ Run `dplyr::last_dplyr_warnings()` to see the 1 remaining warning."
`summarise()` has grouped output by 'gene1', 'gene2', 'ncells'. You can
override using the `.groups` argument.
